In [31]:
def dependensyTree(arg):
    d=dict((k, set(arg[k])) for k in arg)
    # print('dict',d)
    r=[]
    while d:
        # values not in keys (items without dep)
        t = set(i for v in d.values() for i in v)-set(d.keys())
        # and keys without value (items without dep)
        t.update(k for k, v in d.items() if not v)
        # can be done right away
        r.append(t)
        # and cleaned up
        d=dict(((k, v-t) for k, v in d.items() if v))
    return r

if __name__=='__main__':
    buffer = {'0': ['2'], '2': ['1', '3', '6'], '6': ['5', '4']}
    print(dependensyTree(buffer))

[{'5', '3', '4', '1'}, {'6'}, {'2'}, {'0'}]


In [1]:
def reverse(graph):
    r = {}
    for src in graph:
        for (dst,c) in graph[src].items():
            if dst in r:
                r[dst][src] = c
            else:
                r[dst] = { src : c }
    return r

In [3]:
graph = {'0': ['2'], '2': ['1', '3', '6'], '6': ['5', '4']}
reverse(graph)

AttributeError: 'list' object has no attribute 'items'

In [4]:
from heapdict import heapdict
class Graph(object):
    def __init__(self,G):
        self.INF = 9999
        self.G = G
        self.N = len(self.G)
        self.r = 0
        self.Q = heapdict()
        for i in range(self.N):
            self.Q[i] = self.INF
        self.Q[self.r] = 0 
        self.pi = [None] * self.N
        self.pi[self.r] = -1

    def v_in_q(self,v):
        #get the keys in 0
        keys = list(self.Q.keys()) #you can actually do if v in Q
        #check if v in keys
        return v in keys

    def adj(self,u):
        neighbors = []
        for index, values in enumerate(self.G[u]):
            if values != 0:
                neighbors.append(index) 
        return neighbors #list() #return only index

    def prim_adjlist(self):
        while (self.Q):
            #u = extradct_min  (dict has no extract_min; you have to suffer from)
            u = self.Q.popitem()[0]
            # print(f'Vertex : {u}')
            #for v in adj[u]
            for v in self.adj(u):
                # print(v, end = " --> ")
                if self.v_in_q(v) and (self.G[u][v][1] < self.Q[v]):   
                    self.pi[v] = u
                    self.Q[v] = self.G[u][v][1]
        return f'Pi Adjancency List: {self.pi}'

In [11]:
G = {
    0 : [(0,0), (1,2), (2,2), (3,5), (4,3)],
    1 : [(0,2), (1,0), (2,1), (3,4), (4,4)],
    2 : [(0,2), (1,1), (2,0), (3,3), (4,5)],
    3 : [(0,5), (1,4), (2,3), (3,0), (4,9)],
    4 : [(0,3), (1,4), (2,5), (3,9), (4,0)]
}

G = {'0': ['2'], '2': ['1', '3', '6'], '6': ['5', '4']}

# graph = Graph(G)
# graph.prim_adjlist()

In [13]:
G.items()

dict_items([('0', ['2']), ('2', ['1', '3', '6']), ('6', ['5', '4'])])

In [26]:
for src in G.keys():
    for dst in G[src].keys():
        print(dst)

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
def reverse_graph(G):
    '''Return the reversed graph where g[dst][src]=G[src][dst]'''
    g={}
    for src in G.keys():
        for dst in G[src].keys():
            if dst not in g.keys():
                g[dst]={}
            g[dst][src]=G[src][dst]
    return g

In [ ]:
def reverse(graph):
    r = {}
    for src in graph:
        for (dst,c) in graph[src].items():
            if dst in r:
                r[dst][src] = c
            else:
                r[dst] = { src : c }
    return r

In [ ]:
def msa(V, E, r, w):
    """
    Recursive Edmond's algorithm as per Wikipedia's explanation
    Returns a set of all the edges of the minimum spanning arborescence.
    V := set( Vertex(v) )
    E := set( Edge(u,v) )
    r := Root(v)
    w := dict( Edge(u,v) : cost)
    """

    """
    Step 1 : Removing all edges that lead back to the root
    """
    for (u,v) in E.copy():
        if v == r:
            E.remove((u,v))
            w.pop((u,v))

    """
    Step 2 : Finding the minimum incoming edge for every vertex. O(n**2) but okay since it is
    a small sized list
    """
    pi = dict()
    for v in V:
        edges = [edge[0] for edge in E if edge[1] == v]
        if not len(edges):
            continue
        costs = [w[(u,v)] for u in edges]
        pi[v] = edges[costs.index(min(costs))]
    
    """
    Step 3 : Finding cycles in the graph
    """
    cycle_vertex = None
    for v in V:
        if cycle_vertex is not None:
            break
        visited = set()
        next_v = pi.get(v)
        while next_v:
            if next_v in visited:
                cycle_vertex = next_v
                break
            visited.add(next_v)
            next_v = pi.get(next_v)
    
    """
    Step 4 : If there is no cycle, return all the minimum edges pi(v)
    """
    if cycle_vertex is None:
        return set([(pi[v],v) for v in pi.keys()])
    
    """
    Step 5 : Otherwise, all the vertices in the cycle must be identified
    """
    C = set()
    C.add(cycle_vertex)
    next_v = pi.get(cycle_vertex)
    while next_v != cycle_vertex:
        C.add(next_v)
        next_v = pi.get(next_v)
    
    """
    Step 6 : Contracting the cycle C into a new node v_c
    v_c is negative and squared to avoid having the same number
    """
    v_c = -cycle_vertex**2
    V_prime = set([v for v in V if v not in C] + [v_c])
    E_prime = set()
    w_prime = dict()
    correspondance = dict()
    for (u,v) in E:
        if u not in C and v in C:
            e = (u,v_c)
            if e in E_prime:
                if w_prime[e] < w[(u,v)] - w[(pi[v],v)]:
                    continue
            w_prime[e] = w[(u,v)] - w[(pi[v],v)]
            correspondance[e] = (u,v)
            E_prime.add(e)
        elif u in C and v not in C:
            e = (v_c,v)
            if e in E_prime:
                old_u = correspondance[e][0]
                if w[(old_u,v)] < w[(u,v)]:
                    continue
            E_prime.add(e)
            w_prime[e] = w[(u,v)]
            correspondance[e] = (u,v)
        elif u not in C and v not in C:
            e = (u,v)
            E_prime.add(e)
            w_prime[e] = w[(u,v)]
            correspondance[e] = (u,v)
    
    """
    Step 7 : Recursively calling the algorithm again until no cycles are found
    """
    tree = msa(V_prime, E_prime, r, w_prime)
    
    """
    Step 8 : 
    """
    cycle_edge = None
    for (u,v) in tree:
        if v == v_c:
            old_v = correspondance[(u,v_c)][1]
            cycle_edge = (pi[old_v],old_v)
            break
    
    ret = set([correspondance[(u,v)] for (u,v) in tree])
    for v in C:
        u = pi[v]
        ret.add((u,v))
        
    ret.remove(cycle_edge)
    
    return ret